# Azure AI Search z integracją NVIDIA NIM i LlamaIndex

W tym notatniku pokażemy, jak wykorzystać modele AI firmy NVIDIA oraz LlamaIndex do stworzenia zaawansowanego pipeline'u Retrieval-Augmented Generation (RAG). Skorzystamy z dużych modeli językowych i osadzeń NVIDIA, zintegrujemy je z Azure AI Search jako magazynem wektorów i przeprowadzimy RAG, aby poprawić jakość i efektywność wyszukiwania.

## Korzyści
- **Skalowalność**: Wykorzystaj duże modele językowe NVIDIA oraz Azure AI Search do skalowalnego i efektywnego wyszukiwania.
- **Efektywność kosztowa**: Optymalizuj wyszukiwanie i pobieranie danych dzięki wydajnemu magazynowi wektorów oraz technikom hybrydowego wyszukiwania.
- **Wysoka wydajność**: Połącz potężne modele językowe z wyszukiwaniem wektorowym, aby uzyskać szybsze i bardziej precyzyjne odpowiedzi.
- **Jakość**: Utrzymuj wysoką jakość wyszukiwania, opierając odpowiedzi modeli językowych na odpowiednich dokumentach.

## Wymagania wstępne
- 🐍 Python 3.9 lub nowszy
- 🔗 [Azure AI Search Service](https://learn.microsoft.com/azure/search/)
- 🔗 Klucz API NVIDIA do uzyskania dostępu do dużych modeli językowych i osadzeń NVIDIA za pośrednictwem mikroserwisów NVIDIA NIM

## Omówione funkcje
- ✅ Integracja dużych modeli językowych NVIDIA (użyjemy [Phi-3.5-MOE](https://build.nvidia.com/microsoft/phi-3_5-moe))
- ✅ Osadzenia NVIDIA (użyjemy [nv-embedqa-e5-v5](https://build.nvidia.com/nvidia/nv-embedqa-e5-v5))
- ✅ Zaawansowane tryby wyszukiwania w Azure AI Search
- ✅ Indeksowanie dokumentów za pomocą LlamaIndex
- ✅ RAG z wykorzystaniem Azure AI Search i LlamaIndex oraz dużych modeli językowych NVIDIA

Zaczynajmy!


In [ ]:
!pip install azure-search-documents==11.5.1
!pip install --upgrade llama-index
!pip install --upgrade llama-index-core
!pip install --upgrade llama-index-readers-file
!pip install --upgrade llama-index-llms-nvidia
!pip install --upgrade llama-index-embeddings-nvidia
!pip install --upgrade llama-index-postprocessor-nvidia-rerank
!pip install --upgrade llama-index-vector-stores-azureaisearch
!pip install python-dotenv

## Instalacja i Wymagania
Utwórz środowisko Python, używając wersji Python >3.10.

## Pierwsze kroki!


Aby rozpocząć, potrzebujesz `NVIDIA_API_KEY`, aby korzystać z modeli NVIDIA AI Foundation:
1) Utwórz darmowe konto na [NVIDIA](https://build.nvidia.com/explore/discover).
2) Kliknij na wybrany model.
3) W sekcji Input wybierz zakładkę Python, kliknij **Get API Key**, a następnie **Generate Key**.
4) Skopiuj i zapisz wygenerowany klucz jako NVIDIA_API_KEY. Od tego momentu powinieneś mieć dostęp do endpointów.


In [3]:
import getpass
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

if not os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    nvidia_api_key = getpass.getpass("Enter your NVIDIA API key: ")
    assert nvidia_api_key.startswith("nvapi-"), f"{nvidia_api_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvidia_api_key


## Przykład RAG z użyciem LLM i Embedding
### 1) Inicjalizacja LLM
`llama-index-llms-nvidia`, znany również jako konektor LLM od NVIDIA, umożliwia połączenie i generowanie z kompatybilnych modeli dostępnych w katalogu API NVIDIA. Zobacz tutaj listę modeli do uzupełniania rozmów: https://build.nvidia.com/search?term=Text-to-Text

Tutaj użyjemy **mixtral-8x7b-instruct-v0.1**


In [75]:
from llama_index.core import Settings
from llama_index.llms.nvidia import NVIDIA

# Here we are using mixtral-8x7b-instruct-v0.1 model from API Catalog
Settings.llm = NVIDIA(model="microsoft/phi-3.5-moe-instruct", api_key=os.getenv("NVIDIA_API_KEY"))

### 2) Zainicjuj Wbudowanie
`llama-index-embeddings-nvidia`, znany również jako konektor Embeddings od NVIDIA, umożliwia połączenie i generowanie z kompatybilnych modeli dostępnych w katalogu API NVIDIA. Wybraliśmy `nvidia/nv-embedqa-e5-v5` jako model wbudowania. Zobacz tutaj listę modeli wbudowania tekstu: https://build.nvidia.com/nim?filters=usecase%3Ausecase_text_to_embedding%2Cusecase%3Ausecase_image_to_embedding


In [6]:
from llama_index.embeddings.nvidia import NVIDIAEmbedding

Settings.embed_model = NVIDIAEmbedding(model="nvidia/nv-embedqa-e5-v5", api_key=os.getenv("NVIDIA_API_KEY"))

### 3) Utwórz Azure AI Search Vector Store


In [76]:
import logging
import sys
import os
import getpass
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from IPython.display import Markdown, display
from llama_index.vector_stores.azureaisearch import AzureAISearchVectorStore, IndexManagement


search_service_api_key = os.getenv('AZURE_SEARCH_ADMIN_KEY') or getpass.getpass('Enter your Azure Search API key: ')
search_service_endpoint = os.getenv('AZURE_SEARCH_SERVICE_ENDPOINT') or getpass.getpass('Enter your Azure Search service endpoint: ')
search_service_api_version = "2024-07-01"
credential = AzureKeyCredential(search_service_api_key)

# Index name to use
index_name = "llamaindex-nvidia-azureaisearch-demo"

# Use index client to demonstrate creating an index
index_client = SearchIndexClient(
    endpoint=search_service_endpoint,
    credential=credential,
)

# Use search client to demonstrate using existing index
search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=index_name,
    credential=credential,
)

In [ ]:
vector_store = AzureAISearchVectorStore(
    search_or_index_client=index_client,
    index_name=index_name,
    index_management=IndexManagement.CREATE_IF_NOT_EXISTS,
    id_field_key="id",
    chunk_field_key="chunk",
    embedding_field_key="embedding",
    embedding_dimensionality=1024, # dimensionality for nv-embedqa-e5-v5 model
    metadata_string_field_key="metadata",
    doc_id_field_key="doc_id",
    language_analyzer="en.lucene",
    vector_algorithm_type="exhaustiveKnn",
    # compression_type="binary" # Option to use "scalar" or "binary". NOTE: compression is only supported for HNSW
)

In [20]:
from llama_index.core import SimpleDirectoryReader, StorageContext, VectorStoreIndex
from llama_index.core.text_splitter import TokenTextSplitter

# Configure text splitter (nv-embedqa-e5-v5 model has a limit of 512 tokens per input size)
text_splitter = TokenTextSplitter(separator=" ", chunk_size=500, chunk_overlap=10)

# Load documents
documents = SimpleDirectoryReader(
    input_files=["data/txt/state_of_the_union.txt"]
).load_data()
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create index with text splitter
index = VectorStoreIndex.from_documents(
    documents,
    transformations=[text_splitter],
    storage_context=storage_context,
)

### 5) Utwórz silnik zapytań do zadawania pytań na podstawie swoich danych

Oto zapytanie wykorzystujące czyste wyszukiwanie wektorowe w Azure AI Search i osadzające odpowiedź w naszym LLM (Phi-3.5-MOE)


In [69]:
query_engine = index.as_query_engine()
response = query_engine.query("Who did the speaker mention as being present in the chamber?")
display(Markdown(f"{response}"))

 The speaker mentioned the Ukrainian Ambassador to the United States, along with other members of Congress, the Cabinet, and various officials such as the Vice President, the First Lady, and the Second Gentleman, as being present in the chamber.

Oto zapytanie wykorzystujące wyszukiwanie hybrydowe w Azure AI Search.


In [70]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.vector_stores.types import VectorStoreQueryMode
from IPython.display import Markdown, display
from llama_index.core.schema import MetadataMode

# Initialize hybrid retriever and query engine
hybrid_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.HYBRID)
hybrid_query_engine = RetrieverQueryEngine(retriever=hybrid_retriever)

# Query execution
query = "What were the exact economic consequences mentioned in relation to Russia's stock market?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))

 The Russian stock market experienced a significant drop, losing 40% of its value. Additionally, trading had to be suspended due to the ongoing situation.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

building a coalition of other freedom-loving nations from Europe and the Americas to Asia and Africa to confront Putin. 

I spent countless hours unifying our European allies. We shared with the world in advance what we knew Putin was planning and precisely how he would try to falsely justify his aggression.  

We countered Russia’s lies with truth.   

And now that he has acted the free world is holding him accountable. 

Along with twenty-seven members of the European Union including France, Germany, Italy, as well as countries like the United Kingdom, Canada, Japan, Korea, Australia, New Zealand, and many others, even Switzerland. 

We are inflicting pain on Russia and supporting the people of Ukraine. Putin is now isolated from the world more than ever. 

Together with our allies –we are right now enforcing powerful economic sanctions. 

We are cutting off Russia’s largest banks from the international financial system.  



#### Analiza wyszukiwania wektorowego
Odpowiedź LLM trafnie oddaje kluczowe ekonomiczne konsekwencje wspomniane w tekście źródłowym dotyczące rosyjskiego rynku akcji. W szczególności stwierdza, że rosyjski rynek akcji odnotował znaczący spadek, tracąc 40% swojej wartości, a handel został zawieszony z powodu bieżącej sytuacji. Ta odpowiedź dobrze odpowiada informacjom zawartym w źródle, wskazując, że LLM poprawnie zidentyfikował i podsumował istotne szczegóły dotyczące wpływu na rynek akcji w wyniku działań Rosji i nałożonych sankcji.

#### Komentarz do węzłów źródłowych
Węzły źródłowe dostarczają szczegółowego opisu ekonomicznych konsekwencji, z jakimi Rosja musiała się zmierzyć w wyniku międzynarodowych sankcji. Tekst podkreśla, że rosyjski rynek akcji stracił 40% swojej wartości, a handel został zawieszony. Dodatkowo wspomina o innych skutkach ekonomicznych, takich jak dewaluacja rubla oraz szersza izolacja rosyjskiej gospodarki. Odpowiedź LLM skutecznie wyodrębniła kluczowe punkty z tych węzłów, koncentrując się na wpływie na rynek akcji zgodnie z zapytaniem.


Teraz przyjrzyjmy się zapytaniu, w którym wyszukiwanie hybrydowe nie daje dobrze uzasadnionej odpowiedzi:


In [71]:
# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it does mention that the events discussed are happening in the current era and that the actions taken are in response to Putin's aggression. For the precise date, one would need to refer to external sources or historical records.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

our forces are not engaged and will not engage in conflict with Russian forces in Ukraine.  

Our forces are not going to Europe to fight in Ukraine, but to defend our NATO Allies – in the event that Putin decides to keep moving west.  

For that purpose we’ve mobilized American ground forces, air squadrons, and ship deployments to protect NATO countries including Poland, Romania, Latvia, Lithuania, and Estonia. 

As I have made crystal clear the United States and our Allies will defend every inch of territory of NATO countries with the full force of our collective power.  

And we remain clear-eyed. The Ukrainians are fighting back with pure courage. But the next few days weeks, months, will be hard on them.  

Putin has unleashed violence and chaos.  But while he may make gains on the battlefield – he will pay a continuing high price over the long run. 

And a proud Ukrainian people, who have known 30 years  of independence,

### Hybrydowe wyszukiwanie: Analiza odpowiedzi LLM
Odpowiedź LLM w przykładzie hybrydowego wyszukiwania wskazuje, że dostarczony kontekst nie określa dokładnej daty inwazji Rosji na Ukrainę. Ta odpowiedź sugeruje, że LLM korzysta z informacji dostępnych w dokumentach źródłowych, jednocześnie przyznając brak precyzyjnych szczegółów w tekście.

Odpowiedź jest trafna w identyfikacji, że kontekst wspomina o wydarzeniach związanych z agresją Rosji, ale nie wskazuje konkretnej daty inwazji. Pokazuje to zdolność LLM do zrozumienia dostarczonych informacji przy jednoczesnym rozpoznaniu luk w treści. LLM skutecznie zachęca użytkownika do poszukiwania zewnętrznych źródeł lub zapisów historycznych w celu uzyskania dokładnej daty, wykazując ostrożność w sytuacji, gdy informacje są niekompletne.

### Analiza węzłów źródłowych
Węzły źródłowe w przykładzie hybrydowego wyszukiwania zawierają fragmenty przemówienia omawiającego reakcję USA na działania Rosji w Ukrainie. Te węzły podkreślają szerszy wpływ geopolityczny oraz kroki podjęte przez USA i ich sojuszników w odpowiedzi na inwazję, ale nie wspominają o konkretnej dacie inwazji. Jest to zgodne z odpowiedzią LLM, która poprawnie identyfikuje brak precyzyjnych informacji o dacie w kontekście.


In [72]:
# Initialize hybrid retriever and query engine
semantic_reranker_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.SEMANTIC_HYBRID)
semantic_reranker_query_engine = RetrieverQueryEngine(retriever=semantic_reranker_retriever)

# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = semantic_reranker_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it mentions that the event occurred six days before the speech was given. To determine the precise date, one would need to know the date of the speech.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny. 

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. 

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. 

He met the Ukrainian people. 

From President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. 

### Hybrid w/Reranking: Analiza odpowiedzi LLM
W przykładzie Hybrid w/Reranking odpowiedź LLM dostarcza dodatkowego kontekstu, wskazując, że wydarzenie miało miejsce sześć dni przed wygłoszeniem przemówienia. Oznacza to, że LLM potrafi wywnioskować datę inwazji na podstawie czasu przemówienia, choć nadal wymaga znajomości dokładnej daty przemówienia dla precyzji.

Ta odpowiedź pokazuje ulepszoną zdolność do korzystania z wskazówek kontekstowych w celu udzielenia bardziej informacyjnej odpowiedzi. Podkreśla zaletę rerankingu, dzięki któremu LLM może uzyskać dostęp do bardziej istotnych informacji i nadać im priorytet, aby zbliżyć się do pożądanego szczegółu (tj. daty inwazji).

### Analiza węzłów źródłowych
Węzły źródłowe w tym przykładzie zawierają odniesienia do czasu inwazji Rosji, konkretnie wspominając, że miała miejsce sześć dni przed przemówieniem. Chociaż dokładna data nadal nie jest wyraźnie podana, węzły dostarczają kontekstu czasowego, który pozwala LLM udzielić bardziej zniuansowanej odpowiedzi. Uwzględnienie tego szczegółu pokazuje, jak reranking może poprawić zdolność LLM do wydobywania i wnioskowania informacji z dostarczonego kontekstu, prowadząc do bardziej precyzyjnej i informacyjnej odpowiedzi.


**Uwaga:**  
W tym notatniku użyliśmy mikrousług NVIDIA NIM z katalogu API NVIDIA.  
Powyższe API, `NVIDIA (llms)`, `NVIDIAEmbedding` oraz [Azure AI Search Semantic Hybrid Retrieval (wbudowane ponowne rankowanie)](https://learn.microsoft.com/azure/search/semantic-search-overview). Należy pamiętać, że powyższe API mogą również obsługiwać mikrousługi hostowane lokalnie.  

**Przykład:**  
```python
NVIDIA(model="meta/llama3-8b-instruct", base_url="http://your-nim-host-address:8000/v1")```



---

**Zastrzeżenie**:  
Ten dokument został przetłumaczony za pomocą usługi tłumaczeniowej AI [Co-op Translator](https://github.com/Azure/co-op-translator). Chociaż dokładamy wszelkich starań, aby tłumaczenie było precyzyjne, prosimy pamiętać, że automatyczne tłumaczenia mogą zawierać błędy lub nieścisłości. Oryginalny dokument w jego rodzimym języku powinien być uznawany za wiarygodne źródło. W przypadku informacji o kluczowym znaczeniu zaleca się skorzystanie z profesjonalnego tłumaczenia wykonanego przez człowieka. Nie ponosimy odpowiedzialności za jakiekolwiek nieporozumienia lub błędne interpretacje wynikające z użycia tego tłumaczenia.
